In [ ]:
import torch
import os
from dataPreprocessing import *
import pandas as pd
import numpy as np

In [ ]:
RANDOM_SEED = 42
BATCH_SIZE = 64

torch.manual_seed(RANDOM_SEED)

In [ ]:
#folder path were dataset is located
path = 'dataset/'
#initialize empty lists to hold data
train_pos, train_neg, test_pos, test_neg = [], [], [], []
#create a dictionary where the key is the relative path to data and value is empty list
sets_dict = {'train/pos/': train_pos, 'train/neg/': train_neg, 'test/pos/': test_pos, 'test/neg/': test_neg}
#loop through dictionary to read from files and populate empty lists
for dataset in sets_dict:
        file_list = [file for file in os.listdir(os.path.join(path, dataset)) if file.endswith('.txt')]
        file_list = sorted(file_list)
        load_data(os.path.join(path, dataset), file_list, sets_dict[dataset])
#Covert lists to pandas dataframes and combine to form train and test datasets
train_data = pd.concat([pd.DataFrame({'review': train_pos, 'label':1}), pd.DataFrame({'review': train_neg, 'label':0})], axis = 0, ignore_index=True)
test_data = pd.concat([pd.DataFrame({'review': test_pos, 'label':1}), pd.DataFrame({'review': test_neg, 'label':0})], axis = 0, ignore_index=True)

In [ ]:
test_data["tokenized"] = test_data["review"].apply(lambda x: tokenize(clean_text(x.lower())))
#Examine tokenized reviews
print(test_data.head()["tokenized"])

In [ ]:
def pad_or_trim(tokens, length=400, pad_token=""):
    """Pads or trims a list of tokens to a specified length."""
    if len(tokens) > length:
        # Trim the list to the desired length
        return tokens[:length]
    else:
        # Pad the list with the pad_token
        return tokens + [pad_token] * (length - len(tokens))

# Apply the function to create the tokenized column
test_data["tokenized"] = test_data["review"].apply(
    lambda x: pad_or_trim(tokenize(clean_text(x.lower())))
)

# Verify the length of each tokenized review
print(test_data["tokenized"].apply(len).value_counts())

In [ ]:
print(test_data.head()["tokenized"][3])

In [ ]:
# # Load the .pt files
# file_path_a = os.path.join('model_weights', 'ksae_latent_activations_a.pt')
# file_path_b = os.path.join('model_weights', 'ksae_latent_activations_b.pt')
# data_a = torch.load(file_path_a)
# data_b = torch.load(file_path_b)
# # Combine the tensors along the first dimension (dim=0)
# data = torch.cat((data_a, data_b), dim=0)
# print(data.shape)

In [ ]:
file_path = os.path.join('model_weights', 'ksae_latent_activations.pt')
data = torch.load(file_path, map_location='cpu')

In [ ]:
# Reshape the data (25000 reviews, 400 tokens, 64 features)
data = data.reshape(25000,400,64)
print(data.shape)

# Compute the maximum weight for each feature across all tokens (dim=1 aggregates across tokens)
max_weights = data.max(dim=1).values  # Shape: (25000, 64)

# Print the maximum weight for each feature across all tokens
print(max_weights)

# If you want to further summarize it (e.g., max weight per feature across all reviews), you can use:
max_weights_per_feature = max_weights.max(dim=0).values  # Shape: (64,)
print("Maximum weight for each feature across all tokens and reviews:")
print(max_weights_per_feature)

In [ ]:
print(data[24990][10])


In [ ]:
print(data[3][12])

In [ ]:
print(data[3][14])

In [ ]:
def get_max_tokens(feature_number, data, tokenized_data):
    """
    Prints the tokens with the maximum activation for a given feature across all reviews,
    along with their activation values.

    Parameters:
    - feature_number: int, the feature index to analyze (0-63).
    - data: tensor of shape (25000, 400, 64) containing activations.
    - tokenized_data: list of tokenized reviews (each review is a list of 400 tokens).
    """

    # Ensure the feature number is within valid range
    if feature_number < 0 or feature_number >= data.shape[2]:
        raise ValueError(f"Feature number must be between 0 and {data.shape[2] - 1}")

    # Extract the activations for the given feature across all tokens and reviews
    activations = data[:, :, feature_number]  # Shape: (25000, 400)

    # Find the index of the maximum activation per review
    max_token_indices = activations.argmax(dim=1)  # Shape: (25000,)

    # Collect the corresponding tokens with max activation for each review
    max_tokens = [
        (tokenized_data[i][max_token_indices[i].item()], activations[i, max_token_indices[i]].item())
        for i in range(len(tokenized_data))
    ]

    # Print the tokens with the highest activation for the specified feature
    print(f"Tokens with max activation for feature {feature_number}:")
    for i, (token, activation) in enumerate(max_tokens[:50]):  # Show first 10 results for brevity
        print(f"Review {i + 1}: Token = '{token}', Activation = {activation:.4f}")

# Example usage:
# Assume `test_data["tokenized"]` contains the tokenized reviews.
tokenized_data = test_data["tokenized"].tolist()
get_max_tokens(15, data, tokenized_data)  # Replace 0 with the feature number you want to analyze.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def plot_colored_tokens_with_more_spacing(feature_number, review_number, data, tokenized_data):
    """
    Plots tokens with color-coded backgrounds based on their activation values,
    with more spacing between tokens and using a blue-to-red colormap (cool to hot).

    Parameters:
    - feature_number: int, the feature index to analyze (0-63).
    - review_number: int, the review index to analyze (0-24999).
    - data: tensor of shape (25000, 400, 64) containing activations.
    - tokenized_data: list of tokenized reviews (each review is a list of 400 tokens).
    """

    # Ensure feature and review numbers are within valid ranges
    if feature_number < 0 or feature_number >= data.shape[2]:
        raise ValueError(f"Feature number must be between 0 and {data.shape[2] - 1}")
    if review_number < 0 or review_number >= data.shape[0]:
        raise ValueError(f"Review number must be between 0 and {data.shape[0] - 1}")

    # Extract activations for the given feature and review
    activations = data[review_number, :, feature_number].detach().numpy()  # Shape: (400,)

    # Normalize activations for color mapping
    norm = plt.Normalize(activations.min(), activations.max())
    cmap = plt.cm.coolwarm  # Blue to red colormap

    # Create the plot with more spacing
    fig, ax = plt.subplots(figsize=(25, 5))  # Increased width for better spacing

    # Adjust spacing between tokens
    x_positions = np.arange(len(tokenized_data[review_number])) * 10  # Add more space between tokens

    # Display tokens with background colors based on their activation
    for i, token in enumerate(tokenized_data[review_number]):
        color = cmap(norm(activations[i]))
        ax.text(x_positions[i], 0.5, token, ha='center', va='center', fontsize=12,
                bbox=dict(facecolor=color, edgecolor='none', boxstyle='round,pad=0.5'))

    # Set plot limits and remove axis for a cleaner look
    plt.xlim(-1, x_positions[-1] + 1)
    plt.ylim(0, 1)
    plt.axis('off')

    plt.title(f'Token Activation Visualization for Feature {feature_number}, Review {review_number}', fontsize=14)
    plt.show()

# Example usage:
plot_colored_tokens_with_more_spacing(8, 0, data, tokenized_data)  # Replace with your feature and review numbers.


In [ ]:
from IPython.core.display import display, HTML
import matplotlib.cm as cm
import matplotlib.colors as mcolors

def generate_html_colored_tokens(feature_number, review_number, data, tokenized_data):
    """
    Generates HTML to display tokens with color-coded backgrounds based on their activation values.
    Uses a blue-to-red gradient for coloring.

    Parameters:
    - feature_number: int, the feature index to analyze (0-63).
    - review_number: int, the review index to analyze (0-24999).
    - data: tensor of shape (25000, 400, 64) containing activations.
    - tokenized_data: list of tokenized reviews (each review is a list of 400 tokens).
    """

    # Ensure feature and review numbers are within valid ranges
    if feature_number < 0 or feature_number >= data.shape[2]:
        raise ValueError(f"Feature number must be between 0 and {data.shape[2] - 1}")
    if review_number < 0 or review_number >= data.shape[0]:
        raise ValueError(f"Review number must be between 0 and {data.shape[0] - 1}")

    # Extract activations for the given feature and review
    activations = data[review_number, :, feature_number].detach().numpy()  # Shape: (400,)

    # Normalize activations for color mapping
    norm = mcolors.Normalize(activations.min(), activations.max())
    cmap = cm.coolwarm  # Blue-to-red colormap

    # Build the HTML content
    html_content = '<div style="display: flex; flex-wrap: wrap; line-height: 2;">'
    for i, token in enumerate(tokenized_data[review_number]):
        if token:
            color = mcolors.rgb2hex(cmap(norm(activations[i]))[:3])  # Convert to hex color
            html_content += f'<span style="background-color: {color}; padding: 5px 10px; margin: 2px; border-radius: 5px; color:black">{token}</span>'
    html_content += '</div>'

    # Display the HTML content
    display(HTML(html_content))

# Example usage:


In [ ]:
from IPython.core.display import display, HTML
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import torch

def generate_html_top_2_tokens(feature_number, data, tokenized_data):
    """
    Generates HTML to display the top 2 tokens with the highest activations for each review,
    ignoring empty or whitespace-only tokens.
    Uses a blue-to-red gradient for coloring.

    Parameters:
    - feature_number: int, the feature index to analyze (0-63).
    - data: tensor of shape (25000, 400, 64) containing activations.
    - tokenized_data: list of tokenized reviews (each review is a list of 400 tokens).
    """

    # Ensure the feature number is within the valid range
    if feature_number < 0 or feature_number >= data.shape[2]:
        raise ValueError(f"Feature number must be between 0 and {data.shape[2] - 1}")

    # Normalize activations for color mapping
    cmap = cm.coolwarm  # Blue-to-red colormap
    all_activations = data[:, :, feature_number].detach().numpy()  # Extract activations for the feature
    norm = mcolors.Normalize(all_activations.min(), all_activations.max())  # Normalize activations

    # Build the HTML content
    html_content = '<div style="display: flex; flex-wrap: wrap; line-height: 2;">'

    for review_idx, review in enumerate(tokenized_data):
        # Get activations for the current review
        activations = all_activations[review_idx]  # Shape: (400,)

        # Filter out empty or whitespace-only tokens and collect their indices
        valid_tokens = [(i, token) for i, token in enumerate(review) if token.strip()]
        
        # Get the activations corresponding to valid tokens
        valid_activations = [activations[i] for i, _ in valid_tokens]

        # If fewer than 2 valid tokens, skip this review
        if len(valid_activations) < 2:
            continue

        # Find the top 2 valid token indices based on activations
        top_2_indices = sorted(range(len(valid_activations)), key=lambda i: valid_activations[i], reverse=True)[:2]

        # Add the top 2 tokens to the HTML content
        for idx in top_2_indices:
            token_idx, token = valid_tokens[idx]
            activation = valid_activations[idx]
            color = mcolors.rgb2hex(cmap(norm(activation))[:3])  # Convert to hex color
            html_content += f'<span style="background-color: {color}; padding: 5px 10px; margin: 5px; border-radius: 5px; color:black">{token}</span>'

    html_content += '</div>'

    # Display the HTML content
    display(HTML(html_content))

# Example usage:
generate_html_top_2_tokens(8, data[:, 0:len(data)//2], tokenized_data)  # Replace 8 with the desired feature number.


In [ ]:
from IPython.core.display import display, HTML
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import torch

def generate_html_heatmap_top_reviews(feature_number, top_n_reviews, data, tokenized_data):
    """
    Generates an HTML heatmap for all tokens in the top reviews with the highest activations for a specific feature.
    Uses a blue-to-red gradient for coloring.

    Parameters:
    - feature_number: int, the feature index to analyze (0-63).
    - top_n_reviews: int, the number of reviews to display based on highest activation scores.
    - data: tensor of shape (25000, 400, 64) containing activations.
    - tokenized_data: list of tokenized reviews (each review is a list of 400 tokens).
    """

    # Ensure the feature number is within the valid range
    if feature_number < 0 or feature_number >= data.shape[2]:
        raise ValueError(f"Feature number must be between 0 and {data.shape[2] - 1}")

    # Extract activations for the feature
    activations = data[:, :, feature_number].detach().numpy()  # Shape: (25000, 400)

    # Calculate the average activation per review for the specified feature
    avg_activations = activations.mean(axis=1)  # Shape: (25000,)

    # Find the indices of the top reviews with the highest average activation
    top_review_indices = avg_activations.argsort()[-top_n_reviews:][::-1]

    # Normalize activations for color mapping
    cmap = cm.coolwarm  # Blue-to-red colormap
    norm = mcolors.Normalize(activations.min(), activations.max())  # Normalize across all activations

    # Build the HTML content
    html_content = '<div style="display: flex; flex-direction: column; gap: 15px; max-width: 100%; line-height: 4; flex-wrap: wrap;">'

    for review_idx in top_review_indices:
        # Get the tokens and activations for the current top review
        review_tokens = tokenized_data[review_idx]
        review_activations = activations[review_idx]  # Shape: (400,)

        # Filter out empty or whitespace-only tokens and collect their indices
        valid_tokens = [(i, token) for i, token in enumerate(review_tokens) if token.strip()]
        
        # Get the activations corresponding to valid tokens
        valid_activations = [review_activations[i] for i, _ in valid_tokens]

        # Add tokens to the HTML content with color based on activation
        html_content += '<div style="margin-bottom: 10px; line-height: 2.5;">'
        for i, token in enumerate(valid_tokens):
            token_idx, token = token
            activation = valid_activations[i]
            color = mcolors.rgb2hex(cmap(norm(activation))[:3])  # Convert to hex color
            html_content += f'<span style="background-color: {color}; padding: 5px 10px; margin: 5px; border-radius: 5px; color:black; display:inline-block; max-width: 100%; overflow-wrap: break-word;">{token}</span>'
        html_content += '</div>'

    html_content += '</div>'

    # Display the HTML content
    display(HTML(html_content))

# Example usage:
generate_html_heatmap_top_reviews(43, 25, data, tokenized_data)  # Replace 8 with the desired feature number, and 10 with the number of top reviews.


In [ ]:
import torch  # Ensure PyTorch is imported

def get_max_tokens(feature_number, data, tokenized_data, start=0, end=1000):
    """
    Returns the tokens with the maximum activation for a given feature across all reviews,
    along with their activation values, including empty tokens (whitespace-only).

    Parameters:
    - feature_number: int, the feature index to analyze (0-63).
    - data: tensor of shape (25000, 400, 64) containing activations.
    - tokenized_data: list of tokenized reviews (each review is a list of 400 tokens).
    - start: int, the starting index for displaying results (default=0).
    - end: int, the ending index for displaying results (default=1000).

    Returns:
    - List of tuples: (token, activation_value) for each review's highest activated token.
    """

    # Ensure the feature number is within a valid range
    if feature_number < 0 or feature_number >= data.shape[2]:
        raise ValueError(f"Feature number must be between 0 and {data.shape[2] - 1}")

    # Extract the activations for the given feature across all tokens and reviews
    activations = data[:, :, feature_number]  # Shape: (25000, 400)

    # Find the index of the maximum activation per review
    max_token_indices = activations.argmax(dim=1)  # Shape: (25000,)

    # Collect the corresponding tokens with max activation for each review
    max_tokens = [
        (tokenized_data[i][idx.item()], activations[i, idx].item())
        for i, idx in enumerate(max_token_indices)
    ]

    # Display the first 10 results for verification (within the specified range)
    for i, (token, activation) in enumerate(max_tokens[start:end]):
        print(f"{start + i + 1}: Token = '{token}', Activation = {activation:.4f}")

    return max_tokens


In [ ]:

results = get_max_tokens(8, data, tokenized_data,0,12501)  


In [ ]:
results = get_max_tokens(24, data, tokenized_data, 0,12501)  

In [ ]:
results = get_max_tokens(32, data, tokenized_data,0,12501)  

In [ ]:
results = get_max_tokens(36, data, tokenized_data, 0, 12501) 

In [ ]:
results = get_max_tokens(44, data, tokenized_data,0,12501)  

In [ ]:
results = get_max_tokens(47, data, tokenized_data,12499,25000)  


In [ ]:
results = get_max_tokens(54, data, tokenized_data, 0,12501)  

In [ ]:
def print_top_n_tokens_overall(feature_number, data, tokenized_data, n=10):
    """
    Prints the top n tokens with the highest activations across all reviews,
    ignoring empty or whitespace-only tokens.

    Parameters:
    - feature_number: int, the feature index to analyze (0-63).
    - data: tensor of shape (25000, 400, 64) containing activations.
    - tokenized_data: list of tokenized reviews (each review is a list of 400 tokens).
    - n: int, the number of top tokens to print.
    """

    # Ensure the feature number is within the valid range
    if feature_number < 0 or feature_number >= data.shape[2]:
        raise ValueError(f"Feature number must be between 0 and {data.shape[2] - 1}")

    # Extract activations for the specified feature
    all_activations = data[:, :, feature_number].detach().numpy()  # Shape: (25000, 400)

    # Store tokens and activations in a list of tuples (activation, token)
    tokens_activations = []

    for review_idx, review in enumerate(tokenized_data):
        activations = all_activations[review_idx]  # Shape: (400,)

        # Filter out empty or whitespace-only tokens
        valid_tokens = [(i, token) for i, token in enumerate(review) if token.strip()]
        
        # Get the activations corresponding to valid tokens and store them
        tokens_activations.extend([(activations[i], token) for i, token in valid_tokens])

    # Sort the list by activation values in descending order and get the top n tokens
    top_n_tokens = sorted(tokens_activations, key=lambda x: x[0], reverse=True)[:n]

    # Print the top n tokens and their activation values
    print(f"Top {n} tokens overall for feature {feature_number}:")
    for activation, token in top_n_tokens:
        print(f"  Token: '{token}', Activation: {activation:.4f}")

# 15, 26, 43, 44, 48, 49, 52
# Example usage:
print_top_n_tokens_overall(48, data, tokenized_data, n=500)  # Replace 8 with the desired feature number.


# Pipeline for analyzing model

## Helper Functions for analysis

In [105]:
file_path = os.path.join('model_weights', 'ksae_latent_activations.pt')
data = torch.load(file_path, map_location='cpu')

In [107]:
def calculate_global_feature_data(data, tokenized_data, feature_list=[15, 26, 43, 44, 48, 49, 52]):
    """
    Calculates mean non-zero activations for features using vectorized operations.
    
    This optimized version avoids loops by leveraging PyTorch's tensor operations,
    which are implemented in C++ and can utilize GPU acceleration if available.
    
    Parameters:
    - data: Tensor of shape (25000, 400, 64) containing activations
    - tokenized_data: List of tokenized reviews
    - feature_list: List of feature indices to analyze
    
    Returns:
    - feature_means: Dictionary of mean activations per feature
    - reference_list: List of formatted reference strings
    """
    # Convert feature list to tensor for efficient indexing
    feature_indices = torch.tensor(feature_list, device=data.device)
    
    # Create padding mask
    padding_mask = torch.tensor([
        [1 if token != '[PAD]' else 0 for token in review]
        for review in tokenized_data
    ], device=data.device)
    
    # Extract relevant features and apply padding mask
    # Shape: (25000, 400, len(feature_list))
    feature_activations = data[:, :, feature_indices]
    feature_activations = feature_activations * padding_mask.unsqueeze(-1)
    
    # Create non-zero mask (threshold at 0.001)
    active_mask = (torch.abs(feature_activations) >= 0.001).float()
    
    # Calculate sums and counts efficiently
    total_activations = (feature_activations * active_mask).sum(dim=(0, 1))
    activation_counts = active_mask.sum(dim=(0, 1))
    
    # Calculate means (avoiding division by zero)
    means = torch.where(
        activation_counts > 0,
        total_activations / activation_counts,
        torch.zeros_like(total_activations)
    )
    
    # Convert results to Python structures
    feature_means = {
        feature_list[i]: means[i].item() 
        for i in range(len(feature_list))
    }
    
    # Create reference strings with count information
    reference_list = [
        f"Feature {feature_list[i]}: mean activation = {means[i]:.3f} "
        f"(from {int(activation_counts[i].item())} active tokens)"
        for i in range(len(feature_list))
    ]
    
    return feature_means, reference_list

# Example usage:
feature_means, reference_list = calculate_global_feature_data(data, tokenized_data)

print("Global Feature Statistics:")
for ref in reference_list:
    print(ref)
print("\n" + "="*50 + "\n")


IndexError: too many indices for tensor of dimension 2

In [ ]:
def analyze_review(review_index, data, tokenized_data, feature_means, 
                                feature_list=[15, 26, 43, 44, 48, 49, 52]):
    """
    Analyzes a review's feature activations and compares them to the global means.
    This helps us understand when a feature is behaving typically or unusually in a specific review.
    """
    review_activations = data[review_index]
    num_tokens = len(tokenized_data[review_index])
    
    print(f"Analyzing review #{review_index}")
    
    for feature in feature_list:
        non_padding_count = 0
        zero_count = 0
        active_count = 0
        non_zero_activations = []
        
        for position in range(num_tokens):
            if tokenized_data[review_index][position] != '[PAD]':
                activation = review_activations[position, feature].item()
                non_padding_count += 1
                if abs(activation) < 0.001:
                    zero_count += 1
                else:
                    active_count += 1
                    non_zero_activations.append(activation)
        
        if non_zero_activations:
            mean_activation = sum(non_zero_activations) / len(non_zero_activations)
            diff_from_mean = mean_activation - feature_means[feature]
        else:
            mean_activation = 0
            diff_from_mean = -feature_means[feature]
        
        print(f"\nFeature {feature}:")
        print(f"Active tokens: {active_count}/{non_padding_count} ({(active_count/non_padding_count)*100:.1f}%)")
        print(f"Zero tokens: {zero_count}/{non_padding_count} ({(zero_count/non_padding_count)*100:.1f}%)")
        
        if non_zero_activations:
            print(f"Mean activation in review: {mean_activation:.3f}")
            print(f"Global mean activation: {feature_means[feature]:.3f}")
            # print(f"Difference from global: {diff_from_mean:+.3f}")
        else:
            print("No active tokens in this review")
            print(f"Global mean activation: {feature_means[feature]:.3f}")
            
        print("-" * 50)

## Start your analysis here

In [ ]:
# Fill in the review #s you're analyzing
reviews = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [ ]:
# Run this cell for every review you're analyzing
review_idx = 19

analyze_review(review_idx, data, tokenized_data, feature_means)

# Feature 15: Positive
# Feature 26: Negative/Disgust
# Feature 43: Review Structure/Mixed
# Feature 44: Narrative elements
# Feature 48: Film genre
# Feature 49: Film craftsmenship/plot
# Feature 52: Eeriness/eerie environments

Analyzing review #19

Feature 15:
Active tokens: 265/400 (66.2%)
Zero tokens: 135/400 (33.8%)
Mean activation in review: 1.364
Global mean activation: 1.207
--------------------------------------------------

Feature 26:
Active tokens: 135/400 (33.8%)
Zero tokens: 265/400 (66.2%)
Mean activation in review: 1.391
Global mean activation: 1.819
--------------------------------------------------

Feature 43:
Active tokens: 149/400 (37.2%)
Zero tokens: 251/400 (62.7%)
Mean activation in review: 0.766
Global mean activation: 0.764
--------------------------------------------------

Feature 44:
Active tokens: 251/400 (62.7%)
Zero tokens: 149/400 (37.2%)
Mean activation in review: 0.798
Global mean activation: 0.781
--------------------------------------------------

Feature 48:
Active tokens: 400/400 (100.0%)
Zero tokens: 0/400 (0.0%)
Mean activation in review: 0.139
Global mean activation: 0.123
--------------------------------------------------

Feature 49:
Active tokens: 185/400 (46.2%)
Ze

In [ ]:
# Sunidhi's analysis
# Review 12: negative, higher non zero activation for negative than positive, higher mean than global for eeriness, moderate for everything else
# Review 13: negative, negative is closer to global mean, high eerieness mean
# Review 14: positive, 72% active for positive vs 25% for negative, high eeriness but still
# Review 15: postitive, 72% active + review mean higher than global mean
# Review 16: negative, higher active tokens
# Review 17: positive, lower mean for negative, equal mean for positive
# Review 18: positive, same
# Review 19: same

In [ ]:
# fill in your predictions for the 10 reviews
your_predictions = np.array([1, 1, 0, 0, 1, 1, 0, 1, 1, 1])

### Run the following cell after you've filled out ALL your predictions and want to check your accuracy!

In [ ]:
# Load the saved results
results = torch.load('model_weights/test_predictions.pt')

# Access the predictions and ground truth labels
ground_truth = np.array(results['ground_truth'])
model_predictions = np.array(results['predictions'])

In [ ]:
model_preds = model_predictions[reviews]
labels = ground_truth[reviews]
print("Reviews analyzed: ", reviews)
print("Your Predictions: ", your_predictions)
print("Model Predictions: ", model_preds)
print("Ground Truth: ", labels)
print("Your Accuracy against model: ", np.mean(your_predictions==model_preds))
print("Your Accuracy against labels: ", np.mean(your_predictions==labels))
print("Model's accuracy against labels: ", np.mean(model_preds==labels))

Reviews analyzed:  [10 11 12 13 14 15 16 17 18 19]
Your Predictions:  [1 1 0 0 1 1 0 1 1 1]
Model Predictions:  [1 1 0 0 1 0 0 1 1 0]
Ground Truth:  [1 1 0 0 1 0 0 1 1 1]
Your Accuracy against model:  0.8
Your Accuracy against labels:  0.9
Model's accuracy against labels:  0.9
